In [15]:
import os 

from keras.models import Sequential 
from keras.layers import Flatten, Dense, Dropout, Cropping2D, Lambda, InputLayer
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D
from keras import losses, optimizers, metrics, backend, regularizers 
from keras.constraints import maxnorm
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping
from keras.layers.normalization import BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [5]:
from typing import Iterator, List, Union, Tuple
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [6]:
dataset = ('C:/Users/Ketan/Thesis/PilotNet/src/Resized1')

In [7]:
# Read the data
df = pd.io.parsers.read_csv(os.path.join(dataset, 'data_new.csv'))
# Split data into training and validation sets
df_train, df_val = train_test_split(df, test_size=.2)


In [9]:
datagen=ImageDataGenerator(rescale=1./255)

In [10]:
train_generator=datagen.flow_from_dataframe(dataframe=df_train, directory=dataset, x_col="image", y_col="angle", 
                                            class_mode="raw", target_size=(128, 228), batch_size=32)

Found 36324 validated image filenames.


In [11]:
val_generator=datagen.flow_from_dataframe(dataframe=df_val, directory=dataset, x_col="image", y_col="angle", 
                                            class_mode="raw", target_size=(128, 228), batch_size=32)

Found 9081 validated image filenames.


In [16]:
def build_model(drop_prob = 0.2, learn_rate=0.001, constraint=maxnorm(3), reg = None, verbose = False): 
    backend.clear_session()
    model = Sequential()

    ###--- Convolutional Neural Network --- ### 
    input_shape = (128, 228, 3)
    model.add(InputLayer(input_shape=input_shape))

#     model.add(Cropping2D(cropping=((60, 0), (0, 0)), input_shape=input_shape))
    # model.add(Lambda(resize, input_shape=input_shape))
    model.add(BatchNormalization(name='r0'))

    ### conv. layers 
    model.add(Convolution2D(filters=24,
                            kernel_size=(5, 5),
                            strides = (2, 2),
                            activation='relu',
                            kernel_regularizer=reg, 
                            input_shape=input_shape, 
                            ))
    '''READ THIS try adding max pool - before min loss was 7.04'''
#     model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization(name='r1'))

    model.add(Convolution2D(filters=36,
                            kernel_size=(5, 5),
                            strides = (2, 2),
                            activation='relu',
                            kernel_regularizer=reg, 
                            ))
    model.add(BatchNormalization(name='r2'))

    model.add(Convolution2D(filters=48,
                            kernel_size=(5, 5),
                            strides = (2, 2),
                            activation='relu',
                            kernel_regularizer=reg, 
                            ))
    model.add(BatchNormalization(name='r3'))

    model.add(Convolution2D(filters=64,
                            kernel_size=(3, 3),
                            strides = (1, 1),
                            activation='relu',
                            kernel_regularizer=reg, 
                            ))
    model.add(BatchNormalization(name='r4'))

    model.add(Convolution2D(filters=64,
                            kernel_size=(1, 1),
                            strides = (2, 2),
                            activation='relu',
                            kernel_regularizer=reg, 
                            ))
    model.add(BatchNormalization(name='r5'))

    ### fully connect layers 
    model.add(Flatten())
    model.add(Dense(1164, activation='relu', kernel_constraint=constraint))
    model.add(Dropout(drop_prob))

    model.add(Dense(100, activation='relu', kernel_constraint=constraint))
    model.add(Dropout(drop_prob))

    model.add(Dense(50, activation='relu', kernel_constraint=constraint))
    model.add(Dropout(drop_prob))

    model.add(Dense(10, activation='relu', kernel_constraint=constraint))
    model.add(Dropout(drop_prob))

    model.add(Dense(1, activation='linear'))

    if verbose: 
        # Summarize the model
        model.summary()
    
    #compile 
    model.compile(loss='mse', optimizer=optimizers.Adam(lr=learn_rate))

    return model

In [13]:
#tensorboard object 
!mkdir logs5
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=0,
                          write_graph=True, write_images=False)

#save and overwrite models when improved 
filepath="checkpoints/baseline_best.h5"

!mkdir checkpoints5
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', save_best_only=True)

early_stop = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=3, verbose=0, mode='auto')

callbacks_list = [checkpoint, tensorboard]

In [17]:
BATCH_SIZE = 32
EPOCHS = 20
LEARN_RATE = 0.001
DROP_PROB = 0.4

control_model = build_model(drop_prob = DROP_PROB, learn_rate = LEARN_RATE, verbose=True)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
r0 (BatchNormalization)      (None, 128, 228, 3)       12        
_________________________________________________________________
conv2d (Conv2D)              (None, 62, 112, 24)       1824      
_________________________________________________________________
r1 (BatchNormalization)      (None, 62, 112, 24)       96        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 54, 36)        21636     
_________________________________________________________________
r2 (BatchNormalization)      (None, 29, 54, 36)        144       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 25, 48)        43248     
_________________________________________________________________
r3 (BatchNormalization)      (None, 13, 25, 48)        1

C:\Users\Ketan\anaconda3\envs\tf-gpu-cuda\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [18]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=val_generator.n//val_generator.batch_size

In [20]:
control_hist = control_model.fit_generator(generator = train_generator,
                           steps_per_epoch = STEP_SIZE_TRAIN,
                           validation_data = val_generator,
                           validation_steps =STEP_SIZE_VALID, 
                           epochs = EPOCHS,
                           callbacks = callbacks_list,
                           verbose = 1)

C:\Users\Ketan\anaconda3\envs\tf-gpu-cuda\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
1135/1135 [==============================] - 210s 177ms/step - loss: 762.9199 - val_loss: 448.3863
Epoch 2/20
1135/1135 [==============================] - 190s 168ms/step - loss: 572.9327 - val_loss: 384.1129
Epoch 3/20
1135/1135 [==============================] - 191s 168ms/step - loss: 441.0208 - val_loss: 346.7112
Epoch 4/20
1135/1135 [==============================] - 190s 167ms/step - loss: 394.4545 - val_loss: 319.6095
Epoch 5/20
1135/1135 [==============================] - 190s 167ms/step - loss: 350.7538 - val_loss: 313.5581
Epoch 6/20
1135/1135 [==============================] - 191s 168ms/step - loss: 358.4979 - val_loss: 270.4612
Epoch 7/20
1135/1135 [==============================] - 192s 169ms/step - loss: 374.4212 - val_loss: 302.8915
Epoch 8/20
1135/1135 [==============================] - 191s 168ms/step - loss: 347.1401 - val_loss: 257.0193
Epoch 9/20
1135/1135 [==============================] - 190s 167ms/step - loss: 325.9408 - val_loss: 237.9079
Epoch 10/2

In [21]:
control_model.save('resized_nvidia_regression_v1.h5')


In [22]:
import cv2 
import glob